# Practical Exam: Customer Purchase Prediction

RetailTech Solutions is a fast-growing international e-commerce platform operating in over 20 countries across Europe, North America, and Asia. They specialize in fashion, electronics, and home goods, with a unique business model that combines traditional retail with a marketplace for independent sellers.

The company has seen rapid growth. A key part of their success has been their data-driven approach to personalization. However, as they plan their expansion into new markets, they need to improve their ability to predict customer behavior.

Their marketing team wants to predict which customers are most likely to make a purchase based on their browsing behavior.

As an AI Engineer, you will help build this prediction system. Your work will directly impact RetailTech's growth strategy and their goal of increasing revenue.


## Data Description

| Column Name | Criteria |
|------------|----------|
| customer_id | Integer. Unique identifier for each customer. No missing values. |
| time_spent | Float. Minutes spent on website per session. Missing values should be replaced with median. |
| pages_viewed | Integer. Number of pages viewed in session. Missing values should be replaced with mean. |
| basket_value | Float. Value of items in basket. Missing values should be replaced with 0. |
| device_type | String. One of: Mobile, Desktop, Tablet. Missing values should be replaced with "Unknown". |
| customer_type | String. One of: New, Returning. Missing values should be replaced with "New". |
| purchase | Binary. Whether customer made a purchase (1) or not (0). Target variable. |

# Task 1

The marketing team has collected customer session data in `raw_customer_data.csv`, but it contains missing values and inconsistencies that need to be addressed.
Create a cleaned version of the dataframe:

- Start with the data in the file `raw_customer_data.csv`
- Your output should be a DataFrame named `clean_data`
- All column names and values should match the table below.
</br>

| Column Name | Criteria |
|------------|----------|
| customer_id | Integer. Unique identifier for each customer. No missing values. |
| time_spent | Float. Minutes spent on website per session. Missing values should be replaced with median. |
| pages_viewed | Integer. Number of pages viewed in session. Missing values should be replaced with mean. |
| basket_value | Float. Value of items in basket. Missing values should be replaced with 0. |
| device_type | String. One of: Mobile, Desktop, Tablet. Missing values should be replaced with "Unknown". |
| customer_type | String. One of: New, Returning. Missing values should be replaced with "New". |
| purchase | Binary. Whether customer made a purchase (1) or not (0). Target variable. |

In [88]:
# Write your answer to Task 1 here 
import pandas as pd
data = pd.read_csv("raw_customer_data.csv")
data["device_type"].fillna("Unknown",inplace=True)
data["customer_type"].fillna("New",inplace=True)
data["basket_value"].fillna(float(0),inplace=True)
views_mean = data['pages_viewed'].mean()
data['pages_viewed'].fillna(views_mean, inplace=True)
time_median = data['time_spent'].median()
data['time_spent'].fillna(time_median, inplace=True)
clean_data = data.copy()
clean_data["pages_viewed"] = clean_data["pages_viewed"].astype(int)
clean_data["device_type"] = clean_data["device_type"].astype(str)
clean_data["customer_type"] = clean_data["customer_type"].astype(str)
clean_data["purchase"] = clean_data["purchase"].astype(bool)
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   customer_id    500 non-null    int64  
 1   time_spent     500 non-null    float64
 2   pages_viewed   500 non-null    int64  
 3   basket_value   500 non-null    float64
 4   device_type    500 non-null    object 
 5   customer_type  500 non-null    object 
 6   purchase       500 non-null    bool   
dtypes: bool(1), float64(2), int64(2), object(2)
memory usage: 24.1+ KB


# Task 2
The pre-cleaned dataset `model_data.csv` needs to be prepared for our neural network.
Create the model features:

- Start with the data in the file `model_data.csv`
- Scale numerical features (`time_spent`, `pages_viewed`, `basket_value`) to 0-1 range
- Apply one-hot encoding to the categorical features (`device_type`, `customer_type`)
    - The column names should have the following format: variable_name_category_name (e.g., `device_type_Desktop`)
- Your output should be a DataFrame named `model_feature_set`, with all column names from `model_data.csv` except for the columns where one-hot encoding was applied.


In [89]:
# Write your answer to Task 2 here
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

model_data = pd.read_csv("model_data.csv")

nmr_features = ['time_spent', 'pages_viewed', 'basket_value']
ctg_features = ['device_type', 'customer_type']
other = ['customer_id','purchase']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), nmr_features),  # Scale numeric features
        ('cat', OneHotEncoder(), ctg_features)  # Encode categorical features
    ]
)

processed_data = preprocessor.fit_transform(model_data)

model_feature_set = pd.DataFrame(processed_data, columns= nmr_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(ctg_features)))
model_feature_set[other] = model_data[other]

# Task 3

Now that all preparatory work has been done, create and train a neural network that would allow the company to predict purchases.

- Using PyTorch, create a network with:
   - At least one hidden layer with 8 units
   - ReLU activation for hidden layer
   - Sigmoid activation for the output layer
- Using the prepared features in `input_model_features.csv`, train the model to predict purchases. 
- Use the validation dataset `validation_features.csv` to predict new values based on the trained model. 
- Your model should be named `purchase_model` and your output should be a DataFrame named `validation_predictions` with columns `customer_id` and `purchase`. The `purchase` column must be your predicted values.


In [90]:
import torch.nn as nn
import torch
train = pd.read_csv("input_model_features.csv")
test = pd.read_csv("validation_features.csv")

feats = ['time_spent', 'pages_viewed', 'basket_value','device_type_Desktop', 'device_type_Mobile', 'device_type_Tablet','device_type_Unknown', 'customer_type_New', 'customer_type_Returning']
target = 'purchase'

feats_train = torch.tensor(train[feats].values, dtype=torch.float32)
target_train =  torch.tensor(train[target].values, dtype=torch.float32)
feats_test =  torch.tensor(test[feats].values, dtype=torch.float32)

purchase_model = nn.Sequential(
    nn.Linear(9,8),
    nn.ReLU(),
    nn.Linear(8,1),
    nn.Sigmoid()
)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(purchase_model.parameters(), lr=0.01)

num_epochs = 100
for epoch in range(num_epochs):
    outputs = purchase_model(feats_train).squeeze()
    loss = criterion(outputs, target_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
with torch.no_grad():
    purchase_model.eval()
    outputs = purchase_model(feats_test).squeeze()
    predicted = (outputs >= 0.5).float()
    
validation_predictions = pd.DataFrame()
validation_predictions['customer_id'] = test['customer_id']
validation_predictions['purchase'] = predicted.numpy()
validation_predictions.head(20)

Epoch [10/100], Loss: 0.5297
Epoch [20/100], Loss: 0.5078
Epoch [30/100], Loss: 0.5051
Epoch [40/100], Loss: 0.4989
Epoch [50/100], Loss: 0.4953
Epoch [60/100], Loss: 0.4917
Epoch [70/100], Loss: 0.4882
Epoch [80/100], Loss: 0.4844
Epoch [90/100], Loss: 0.4807
Epoch [100/100], Loss: 0.4776


,customer_id,purchase
0,1801,1.0
1,1802,1.0
2,1803,1.0
3,1804,1.0
4,1805,1.0
5,1806,1.0
6,1807,1.0
7,1808,1.0
8,1809,1.0
9,1810,1.0
